In [79]:
import re
import json
import requests
from bs4 import BeautifulSoup

In [80]:
headers = json.loads(r'''{
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Cookie": "trctestcookie=ok; trctestcookie=ok; bblastvisit=1517316262; bblastactivity=0; __utmz=60066354.1517316266.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __gads=ID=0150c2db39515748:T=1517316267:S=ALNI_MaDEROky928259c08-Wiv9vdVzIMA; trc_cookie_storage=taboola%2520global%253Auser-id%3D9fb09c65-d038-47cc-aca3-9186c0df7852-tuct1686a87; __utma=60066354.2080145337.1517316266.1519810697.1519820804.11; __utmc=60066354; __utmt=1; __utmb=60066354.123.0.1519822641336",
    "Host": "www.dogforum.com",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36"
}''')

### 試做某一頁，並用Dictionary來儲存list達到去重(sticky每頁都重複)的效果

In [3]:
url = "http://www.dogforum.com/dog-health/3/"

In [4]:
resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text, 'lxml')

In [5]:
soup.select('table')[11].select('#thread_title_331474')[0]

<a href="http://www.dogforum.com/dog-health/potential-pyometra-331474/" id="thread_title_331474">Potential Pyometra</a>

In [6]:
soup.select('table')[11].findAll("a", id=re.compile("^thread_title_"))[0].text

'Members - READ BEFORE POSTING IN HEALTH'

In [7]:
soup.select('table')[11].findAll("a", id=re.compile("^thread_title_"))[0]['href']

'http://www.dogforum.com/dog-health/members-read-before-posting-health-321786/'

In [8]:
thread = {}
for num in range(1,250):
    try:
        key = soup.select('table')[11].findAll("a", id=re.compile("^thread_title_"))[num].text
        value = soup.select('table')[11].findAll("a", id=re.compile("^thread_title_"))[num]['href']
        thread[key] = value
    except IndexError:
        break

In [10]:
thread

{'"What is Parvo?"': 'http://www.dogforum.com/dog-health/what-parvo-5256/',
 '$700 Tab For A Normal De-Scaling': 'http://www.dogforum.com/dog-health/700-tab-normal-de-scaling-325529/',
 '10 days old puppy bloated?': 'http://www.dogforum.com/dog-health/10-days-old-puppy-bloated-331522/',
 '16+ yr old dog with nipple growth': 'http://www.dogforum.com/dog-health/16-yr-old-dog-nipple-growth-328617/',
 '2 weeks of diarrhea with puppy!': 'http://www.dogforum.com/dog-health/2-weeks-diarrhea-puppy-312289/',
 '20 Crate Rest Activities': 'http://www.dogforum.com/dog-health/20-crate-rest-activities-51130/',
 '4-Month Old Rhodesian with Unknown Skin Issue': 'http://www.dogforum.com/dog-health/4-month-old-rhodesian-unknown-skin-327249/',
 '5 year old toy poodle shaking and constantly running away': 'http://www.dogforum.com/dog-health/5-year-old-toy-poodle-shaking-329761/',
 '7mo female dog seizures while licking herself??': 'http://www.dogforum.com/dog-health/7mo-female-dog-seizures-while-licking-3

### 求方便，先手動設定頁數範圍，之後再參考Mobile 01的作法

In [81]:
def board_urls(board_name,last_page):
    thread = {}
    for page in range(1,last_page+1):
        url = "http://www.dogforum.com/{}/{}/".format(board_name, page)
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.text, 'lxml')
        for num in range(1,250):
            try:
                key = soup.select('table')[11].findAll("a", id=re.compile("^thread_title_"))[num].text
                value = soup.select('table')[11].findAll("a", id=re.compile("^thread_title_"))[num]['href']
                thread[key] = value
            except IndexError:
                break
    return thread

In [82]:
board_urls('dog-health',2)

{'"Indoor dogs can\'t get heartworm" - MYTH': 'http://www.dogforum.com/dog-health/indoor-dogs-cant-get-heartworm-myth-334434/',
 '"What is Parvo?"': 'http://www.dogforum.com/dog-health/what-parvo-5256/',
 '(Pics) Pitbull seasonal allergies, food allergies, or mites?': 'http://www.dogforum.com/dog-health/pics-pitbull-seasonal-allergies-food-allergies-331874/',
 '1 ear drooping': 'http://www.dogforum.com/dog-health/1-ear-drooping-349130/',
 '10 Week Old Toy Poodle Fur Concern': 'http://www.dogforum.com/dog-health/10-week-old-toy-poodle-fur-336338/',
 '11 week old puppy, allergies?': 'http://www.dogforum.com/dog-health/11-week-old-puppy-allergies-349170/',
 '11 y/o large mammary mass -running out of options': 'http://www.dogforum.com/dog-health/11-y-o-large-mammary-mass-350042/',
 '12 year old Shih Tzu will be getting Spay': 'http://www.dogforum.com/dog-health/12-year-old-shih-tzu-will-339178/',
 '12 yr old English lab...is it time?': 'http://www.dogforum.com/dog-health/12-yr-old-english-

## All urls in certain board has done!!!  開始抓內文

In [157]:
url = 'http://www.dogforum.com/dog-health/antifreeze-poisoning-7636/'
resp = requests.get(url, headers=headers)
resp.encoding = 'cp1252'
soup = BeautifulSoup(resp.text, 'lxml')

In [158]:
title = soup.find('title').text
title

' Antifreeze poisoning'

In [159]:
# id的re寫法
soup.findAll(id=re.compile("^post"))[2]

<table align="center" border="0" cellpadding="6" cellspacing="0" class="tborder vbseo_like_postbit" id="post84846" width="100%"> <tr> <td class="thead" style="font-weight:normal; border: 1px solid #FCF5DD; border-right: 0px"> <a name="post84846"><img alt="Old" border="0" class="inlineimg" src="http://www.dogforum.com/images/litearth/statusicon/post_old.gif"/></a>
			10-03-2010, 02:42 PM
			
			 </td> <td align="right" class="thead" style="font-weight:normal; border: 1px solid #FCF5DD; border-left: 0px"> <div style="float:left;"> <iframe allowtransparency="true" frameborder="0" scrolling="no" src="http://www.facebook.com/plugins/like.php?href=http%3A%2F%2Fwww.dogforum.com%2Fshowpost.php%3Fp%3D84846&amp;ref=84846&amp;layout=button_count&amp;show_faces=false&amp;width=300&amp;action=like&amp;font=tahoma&amp;colorscheme=light" style="border:none; overflow:hidden; width:300px; height:23px"></iframe> </div> 


			 
			#<a href="http://www.dogforum.com/dog-health/antifreeze-poisoning-7636/#po

In [15]:
soup.select('.tborder.vbseo_like_postbit')[0]

<table align="center" border="0" cellpadding="6" cellspacing="0" class="tborder vbseo_like_postbit" id="post3454482" width="100%"> <tr> <td class="thead" style="font-weight:normal; border: 1px solid #FCF5DD; border-right: 0px"> <a name="post3454482"><img alt="Old" border="0" class="inlineimg" src="http://www.dogforum.com/images/litearth/statusicon/post_old.gif"/></a>
			10-27-2017, 06:15 PM
			
			 </td> <td align="right" class="thead" style="font-weight:normal; border: 1px solid #FCF5DD; border-left: 0px"> <div style="float:left;"> <iframe allowtransparency="true" frameborder="0" scrolling="no" src="http://www.facebook.com/plugins/like.php?href=http%3A%2F%2Fwww.dogforum.com%2Fshowpost.php%3Fp%3D3454482&amp;ref=3454482&amp;layout=button_count&amp;show_faces=false&amp;width=300&amp;action=like&amp;font=tahoma&amp;colorscheme=light" style="border:none; overflow:hidden; width:300px; height:23px"></iframe> </div> 


			 
			#<a href="http://www.dogforum.com/dog-health/ladybugs-339090/#post

### Get post_date 

In [160]:
date = soup.select('.tborder.vbseo_like_postbit')[0].find('td').text.split()[0]
date

'10-03-2010,'

In [112]:
time = soup.select('.tborder.vbseo_like_postbit')[0].find('td').text.split()[1]
time

'02:42'

In [113]:
ampm = soup.select('.tborder.vbseo_like_postbit')[0].find('td').text.split()[2]
ampm

'PM'

In [114]:
post_date = {}
data = {}
data['date'] = soup.select('.tborder.vbseo_like_postbit')[0].find('td').text.split()[0]
data['time'] = soup.select('.tborder.vbseo_like_postbit')[0].find('td').text.split()[1]
data['ampm'] = soup.select('.tborder.vbseo_like_postbit')[0].find('td').text.split()[2]
post_date['post_date'] = data
post_date

{'post_date': {'ampm': 'PM', 'date': '10-03-2010,', 'time': '02:42'}}

### Get post_stories

In [115]:
story = soup.select('.tborder.vbseo_like_postbit')[0].findAll(id=re.compile("^postcount"))[0].get('name')
story

'1'

### Find member detail (name, url member_type)

In [116]:
author_url = soup.select('.tborder.vbseo_like_postbit')[0].select('a.bigusername')[0]['href']
author_url

'http://www.dogforum.com/members/1298/'

In [117]:
author_name = soup.select('.tborder.vbseo_like_postbit')[0].select('a.bigusername')[0].text
author_name

'ScentHound'

In [118]:
member_type = soup.select('.tborder.vbseo_like_postbit')[0].select('div.smallfont')[0].text
member_type

'Senior Member'

In [119]:
soup.select('.tborder.vbseo_like_postbit')[0].select('div.smallfont')[2].text

'\r\n\t\t\t\t\xa0 Join Date: Sep 2009 \r\n\t\t\t\t\tPosts: 11,207\r\n\t\t\t\t Mentioned: 116 Post(s) Tagged: 2 Thread(s)   '

In [120]:
join_date = re.findall('Join Date:(.*)', soup.select('.tborder.vbseo_like_postbit')[0].text)[0].strip()
join_date

'Sep 2009'

In [121]:
posts = re.findall('Posts: (.*)', soup.select('.tborder.vbseo_like_postbit')[0].text)[0].strip()
posts

'11,207'

In [122]:
mentioned = re.findall('Mentioned: (.*)Tag', soup.select('.tborder.vbseo_like_postbit')[0].text)[0].strip()
mentioned

'116 Post(s)'

In [123]:
tagged = re.findall('Tagged: (\d*)', soup.select('.tborder.vbseo_like_postbit')[0].text)[0].strip() + ' thread(s)'
tagged

'2 thread(s)'

### Content

In [124]:
soup.select('.tborder.vbseo_like_postbit')[0].findAll(id=re.compile("^post_message_"))

[<div id="post_message_84846"> <div style="float:right; padding-left:5px; padding-bottom:5px;"> <script type="text/javascript">
 GA_googleFillSlot("Advertising_300x250");
 </script> </div> <!-- google_ad_section_start -->For those that didn't know.<br/> <br/>
 From: <a href="http://www.vetmed.wsu.edu/cliented/antifreeze.aspx" rel="nofollow" target="_blank">Antifreeze Poisoning</a><br/> <br/> <b>Antifreeze Poisoning</b> <br/> <br/>
 This information is not meant to be a substitute for veterinary care. Always follow the instructions provided by your veterinarian.  <br/>
 As winter approaches, many people will "winterize" their automobiles, including a change of antifreeze. Take care to keep both new and used antifreeze in a sealed container, out of reach of pets. Clean up any spills of antifreeze on driveways and other hard surfaces. Dogs and cats find antifreeze quite tasty and if they find antifreeze they'll drink it. Antifreeze is extremely toxic causing kidney failure that is often f

In [30]:
soup.select('.tborder.vbseo_like_postbit')[0].findAll(id=re.compile("^post_message_"))[0].text

'  \r\nGA_googleFillSlot("Advertising_300x250");\r\n  Watch for ladybugs in your dogs mouth. It can cause weird symptoms. Lethargic, foaming at the mouth, and them not eating. It\'s kind of a gross thing. Check your dogs mouth. A positive it\'s a very rare thing. :-) https://www.google.com/amp/s/www.thi...mouth-is-real/  \nSent from my Pixel using Tapatalk '

In [31]:
# use Regex, but still can't purify content
re.findall('google_ad_section_start -->(.*)',str(soup.select('.tborder.vbseo_like_postbit')[0].findAll(id=re.compile("^post_message_"))))

['Watch for ladybugs in your dogs mouth. It can cause weird symptoms. Lethargic, foaming at the mouth, and them not eating. It\'s kind of a gross thing. Check your dogs mouth. A positive it\'s a very rare thing. :-)<br/> <a href="https://www.google.com/amp/s/www.thisdogslife.co/no-its-not-a-hoax-the-photo-of-dozens-of-ladybugs-in-a-dogs-mouth-is-real/amp/#ampshare=https://www.thisdogslife.co/no-its-not-a-hoax-the-photo-of-dozens-of-ladybugs-in-a-dogs-mouth-is-real/" rel="nofollow" target="_blank">https://www.google.com/amp/s/www.thi...mouth-is-real/</a><br/> <img alt="" border="0" src="http://uploads.tapatalk-cdn.com/20171027/c8a506f6cc6ac955ef1586bb781aec7a.jpg"/><br/> <br/>']

In [153]:
url = 'http://www.dogforum.com/dog-health/i-think-my-dog-sick-345913/'
resp = requests.get(url, headers=headers)
resp.encoding = 'cp1252'
soup = BeautifulSoup(resp.text, 'lxml')

In [154]:
# The reason is that two div in this basket, remove first div can reamin main content
a = soup.select('.tborder.vbseo_like_postbit')[0].find("div", id=re.compile("^post_message_"))
a

<div id="post_message_3520681"> <div style="float:right; padding-left:5px; padding-bottom:5px;"> <script type="text/javascript">
GA_googleFillSlot("Advertising_300x250");
</script> </div> <!-- google_ad_section_start -->I have a bichon/Maltese he’s 10 months and we’ve noticed that he isn’t eating all his food (dry nuts). So we started to give him wet food mixed with dry nuts once a day, the past couple of days he’s had diarrhea. He’s also usually very playful and excited. Constantly! He always wants to play, however the past couple of days he’s quite and not playful, he comes to lie down and cuddle but no playing, barking or licking us as he usually does. Not sure if he’s sick. Should I bring him to the vet?<!-- google_ad_section_end --> </div>

In [134]:
a.select('div')[0].extract()

<div style="float:right; padding-left:5px; padding-bottom:5px;"> <script type="text/javascript">
GA_googleFillSlot("Advertising_300x250");
</script> </div>

In [155]:
a.text

'  \r\nGA_googleFillSlot("Advertising_300x250");\r\n  I have a bichon/Maltese he’s 10 months and we’ve noticed that he isn’t eating all his food (dry nuts). So we started to give him wet food mixed with dry nuts once a day, the past couple of days he’s had diarrhea. He’s also usually very playful and excited. Constantly! He always wants to play, however the past couple of days he’s quite and not playful, he comes to lie down and cuddle but no playing, barking or licking us as he usually does. Not sure if he’s sick. Should I bring him to the vet? '

In [151]:
# remember that other stories could not has this issues. Use try except to deal with it
a = soup.select('.tborder.vbseo_like_postbit')[1].find("div", id=re.compile("^post_message_"))
a.select('div')[0].extract()
a.text

IndexError: list index out of range

In [131]:
try:
    content = soup.select('.tborder.vbseo_like_postbit')[2].find("div", id=re.compile("^post_message_"))
    content.select('div')[2].extract()
    content.text
except IndexError:
    content = soup.select('.tborder.vbseo_like_postbit')[2].find("div", id=re.compile("^post_message_")).text
content

' This needs to be a sticky IMO  '

### A better way to prevent unnecessary try except behavior --> if else

In [156]:
a = soup.select('.tborder.vbseo_like_postbit')[0].find("div", id=re.compile("^post_message_"))
if len(a.select('div')) > 0:
    a.select('div')[0].extract()
    content = a.text.strip()
else:
    content = soup.select('.tborder.vbseo_like_postbit')[0].find("div", id=re.compile("^post_message_")).text.strip()
print(content)

I have a bichon/Maltese he’s 10 months and we’ve noticed that he isn’t eating all his food (dry nuts). So we started to give him wet food mixed with dry nuts once a day, the past couple of days he’s had diarrhea. He’s also usually very playful and excited. Constantly! He always wants to play, however the past couple of days he’s quite and not playful, he comes to lie down and cuddle but no playing, barking or licking us as he usually does. Not sure if he’s sick. Should I bring him to the vet?


## Starting auto-distinguish pages of each title

In [44]:
url = 'http://www.dogforum.com/dog-health/what-parvo-5256/'
resp = requests.get(url, headers=headers)
last_page = re.findall("Page 1 of (\d*)", resp.text)[0]
last_page

'9'

In [57]:
url = 'http://www.dogforum.com/dog-health/ladybugs-339090/'
resp = requests.get(url, headers=headers)
last_page = int(re.findall("Page 1 of (\d*)", resp.text)[0])
last_page

3

In [63]:
for page in range(2, last_page+1):
    urls = url+'page%s/'%page
    print(urls)

http://www.dogforum.com/dog-health/ladybugs-339090/page2/
http://www.dogforum.com/dog-health/ladybugs-339090/page3/


In [45]:
# only one page
url = 'http://www.dogforum.com/dog-health/glucosamine-joint-health-350506/'
resp = requests.get(url, headers=headers)
last_page = re.findall("Page 1 of (\d*)", resp.text)[0]
last_page

IndexError: list index out of range

In [66]:
# 法1
url = 'http://www.dogforum.com/dog-health/ladybugs-339090/'
resp = requests.get(url, headers=headers)
try:
    last_page = int(re.findall("Page 1 of (\d*)", resp.text)[0])
    for page in range(1, last_page+1):
        urls = url+'page%s/'%page
        print(urls)
except IndexError:
    urls = url
    print(urls)

http://www.dogforum.com/dog-health/ladybugs-339090/page1/
http://www.dogforum.com/dog-health/ladybugs-339090/page2/
http://www.dogforum.com/dog-health/ladybugs-339090/page3/


In [73]:
# 法2
url = 'http://www.dogforum.com/dog-health/ladybugs-339090/'
resp = requests.get(url, headers=headers)
if not re.findall("Page 1 of (\d*)", resp.text):
    urls = url
    print(urls)
else:
    last_page = int(re.findall("Page 1 of (\d*)", resp.text)[0])
    for page in range(1, last_page+1):
        urls = url+'page%s/'%page
        print(urls)

http://www.dogforum.com/dog-health/ladybugs-339090/page1/
http://www.dogforum.com/dog-health/ladybugs-339090/page2/
http://www.dogforum.com/dog-health/ladybugs-339090/page3/


In [75]:
def get_content_pgurl(url):
    resp = requests.get(url, headers=headers)
    if not re.findall("Page 1 of (\d*)", resp.text):
        urls = url
        return urls
    else:
        last_page = int(re.findall("Page 1 of (\d*)", resp.text)[0])
        urls = [url+'page%s/'%page for page in range(1, last_page+1)]
        return urls

In [78]:
url = 'http://www.dogforum.com/dog-health/ladybugs-339090/'
get_content_pgurl(url)

['http://www.dogforum.com/dog-health/ladybugs-339090/page1/',
 'http://www.dogforum.com/dog-health/ladybugs-339090/page2/',
 'http://www.dogforum.com/dog-health/ladybugs-339090/page3/']

# 開始抓動態樓層數

In [165]:
url = 'http://www.dogforum.com/dog-health/obsessive-licking-cause-345793/'
resp = requests.get(url, headers=headers)
resp.encoding = 'cp1252'
soup = BeautifulSoup(resp.text, 'lxml')

block = {}
post_date = {}

for num in range(1,11):
    try:
        block['title'] = soup.find('title').text
        block['story'] = soup.select('.tborder.vbseo_like_postbit')[num].findAll(id=re.compile("^postcount"))[0].get('name')
        block['author_url'] = soup.select('.tborder.vbseo_like_postbit')[num].select('a.bigusername')[0]['href']
        block['author_name'] = soup.select('.tborder.vbseo_like_postbit')[num].select('a.bigusername')[0].text

        data = {}
        data['date'] = soup.select('.tborder.vbseo_like_postbit')[num].find('td').text.split()[0]
        data['time'] = soup.select('.tborder.vbseo_like_postbit')[num].find('td').text.split()[1]
        data['ampm'] = soup.select('.tborder.vbseo_like_postbit')[num].find('td').text.split()[2]
        block['post_date'] = data

        author = {}
        try:
            author['member_type'] = soup.select('.tborder.vbseo_like_postbit')[num].select('div.smallfont')[0].text
        except IndexError:
            pass
        try:
            author['join_date'] = re.findall('Join Date:(.*)', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip()
        except IndexError:
            pass
        try:
            author['posts'] = re.findall('Posts: (.*)', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip()
        except IndexError:
            pass
        try:
            author['mentioned'] = re.findall('Mentioned: (.*)Tag', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip()
        except IndexError:
            pass
        try:
            author['tagged'] = re.findall('Tagged: (\d*)', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip() + ' thread(s)'
        except IndexError:
            pass
        block['author_info'] = author

        a = soup.select('.tborder.vbseo_like_postbit')[num].find("div", id=re.compile("^post_message_"))
        if len(a.select('div')) > 0:
            a.select('div')[0].extract()
            content = a.text.strip()
        else:
            content = soup.select('.tborder.vbseo_like_postbit')[num].find("div", id=re.compile("^post_message_")).text.strip()
        block['content'] = content.replace("\r","").replace("\n","")
        print(block)
    except IndexError:
        break

{'title': ' Obsessive licking- cause??', 'story': '2', 'author_url': 'http://www.dogforum.com/members/200562/', 'author_name': 'Olympia', 'post_date': {'date': '01-09-2018,', 'time': '05:47', 'ampm': 'AM'}, 'author_info': {'member_type': 'Senior Member', 'join_date': 'Oct 2017', 'posts': '136', 'mentioned': '6 Post(s)', 'tagged': '0 thread(s)'}, 'content': "Ok, so there could be many reasons why she's obsessively licking and chewing, You really need to call your vet and talk to them about the situation, they will determine whether or not you need to bring her in, just going by what you said about blisters on her face, it sounds really bad!! How much exercise does she get each day?What kind of food do you feed her?When she was allowed on the couch did she have access to a window so she could see outside?Is she on flea and tick meds? if so which one?"}
{'title': ' Obsessive licking- cause??', 'story': '3', 'author_url': 'http://www.dogforum.com/members/16965/', 'author_name': 'naturalfed